# Report dataset

This notebook goes over creating the dataset that is used in the AADF vs GDP growth traffic flow report.

It differs from the other dataset as this uses the dataset with added economic statstics

In [1]:
import os
import sys
import importlib

import pandas as pd
import numpy as np

#machine learning
from sklearn.preprocessing import StandardScaler

#table display
from IPython.core.display import HTML

#files for data munging
os.chdir("/home/eddr/Documents/Projects/report")
sys.path.append("/home/eddr/Documents/Projects/report/scripts/")

#custom modules
import dataset_processing as dat


Get the traffic flow aggregated statistics dataset

In [2]:
stats_df = dat.drop_duplicate_rows(dat.load_stats(name =  "traffic_eco_stats.csv",
                                                  loc = os.getcwd() + "/data/",
                                                  format_year = False
                                                 )
                                  )

HTML(stats_df.head().to_html())

,vehicle_measure,count,mean,std,min,25%,50%,75%,max,area_name,area_level,rcat,gdp,gdp_unit,gdp_growth,gdp_ratio,year_label,year,CPIH,CPI,Unemployment (%),average_weekly_earnings,av_weekly_earnings_change,unemployment_pct_change
0,fd2wmv,6102.0,99.717961,98.788596,0.0,39.00,77.0,127.00,1447.0,nat,nuts1,PR,1729121.0,million £,1.5,17292.21,2011,2011,3.8,4.5,8.1,455.0,11.0,0.2
1,fdall_mv,6102.0,11554.632907,9679.563915,82.0,5141.25,9098.0,15269.25,96689.0,nat,nuts1,PR,1729121.0,million £,1.5,17292.21,2011,2011,3.8,4.5,8.1,455.0,11.0,0.2
2,fdbus,6102.0,83.035398,76.557636,0.0,34.00,62.0,108.00,1026.0,nat,nuts1,PR,1729121.0,million £,1.5,17292.21,2011,2011,3.8,4.5,8.1,455.0,11.0,0.2
3,fdcar,6102.0,9335.433956,7941.962675,49.0,4013.25,7275.0,12380.00,81342.0,nat,nuts1,PR,1729121.0,million £,1.5,17292.21,2011,2011,3.8,4.5,8.1,455.0,11.0,0.2
4,fdhgv,6102.0,518.935595,570.248715,4.0,174.00,351.0,644.00,8655.0,nat,nuts1,PR,1729121.0,million £,1.5,17292.21,2011,2011,3.8,4.5,8.1,455.0,11.0,0.2


This looks ok, but bokeh, which we want to use for the interactive charts would prefer wide datasets, so we use this script to turn the tall key-value pair datasets into a wide dataset

In [3]:
stats_unmelted_df = dat.unmelt_by_col(stats_df, 
                                      key_col = "vehicle_measure",
                                      value_cols = ["area_name",
                                                    "area_level",
                                                    "year",
                                                    "rcat",
                                                    "gdp",
                                                    "gdp_growth",
                                                    "gdp_unit",
                                                    "gdp_ratio",
                                                    "year",
                                                    "year_label",
                                                    "CPIH",
                                                    "CPI",
                                                    "Unemployment (%)",
                                                    "average_weekly_earnings",
                                                    "av_weekly_earnings_change",
                                                    "unemployment_pct_change"
                                                   ],
                                      messages = False
                                     )

In [4]:
HTML(stats_unmelted_df.head().to_html())

,vehicle_measure_fd2wmv,count_fd2wmv,mean_fd2wmv,std_fd2wmv,min_fd2wmv,25%_fd2wmv,50%_fd2wmv,75%_fd2wmv,max_fd2wmv,area_name,area_level,rcat,gdp,gdp_unit,gdp_growth,gdp_ratio,year_label,year,CPIH,CPI,Unemployment (%),average_weekly_earnings,av_weekly_earnings_change,unemployment_pct_change,vehicle_measure_fdall_mv,count_fdall_mv,mean_fdall_mv,std_fdall_mv,min_fdall_mv,25%_fdall_mv,50%_fdall_mv,75%_fdall_mv,max_fdall_mv,vehicle_measure_fdbus,count_fdbus,mean_fdbus,std_fdbus,min_fdbus,25%_fdbus,50%_fdbus,75%_fdbus,max_fdbus,vehicle_measure_fdcar,count_fdcar,mean_fdcar,std_fdcar,min_fdcar,25%_fdcar,50%_fdcar,75%_fdcar,max_fdcar,vehicle_measure_fdhgv,count_fdhgv,mean_fdhgv,std_fdhgv,min_fdhgv,25%_fdhgv,50%_fdhgv,75%_fdhgv,max_fdhgv,vehicle_measure_fdhgva3,count_fdhgva3,mean_fdhgva3,std_fdhgva3,min_fdhgva3,25%_fdhgva3,50%_fdhgva3,75%_fdhgva3,max_fdhgva3,vehicle_measure_fdhgva5,count_fdhgva5,mean_fdhgva5,std_fdhgva5,min_fdhgva5,25%_fdhgva5,50%_fdhgva5,75%_fdhgva5,max_fdhgva5,vehicle_measure_fdhgva6,count_fdhgva6,mean_fdhgva6,std_fdhgva6,min_fdhgva6,25%_fdhgva6,50%_fdhgva6,75%_fdhgva6,max_fdhgva6,vehicle_measure_fdhgvr2,count_fdhgvr2,mean_fdhgvr2,std_fdhgvr2,min_fdhgvr2,25%_fdhgvr2,50%_fdhgvr2,75%_fdhgvr2,max_fdhgvr2,vehicle_measure_fdhgvr3,count_fdhgvr3,mean_fdhgvr3,std_fdhgvr3,min_fdhgvr3,25%_fdhgvr3,50%_fdhgvr3,75%_fdhgvr3,max_fdhgvr3,vehicle_measure_fdhgvr4,count_fdhgvr4,mean_fdhgvr4,std_fdhgvr4,min_fdhgvr4,25%_fdhgvr4,50%_fdhgvr4,75%_fdhgvr4,max_fdhgvr4,vehicle_measure_fdlgv,count_fdlgv,mean_fdlgv,std_fdlgv,min_fdlgv,25%_fdlgv,50%_fdlgv,75%_fdlgv,max_fdlgv,vehicle_measure_fdpc,count_fdpc,mean_fdpc,std_fdpc,min_fdpc,25%_fdpc,50%_fdpc,75%_fdpc,max_fdpc,vehicle_measure_fd2wmv_ratio,count_fd2wmv_ratio,mean_fd2wmv_ratio,std_fd2wmv_ratio,min_fd2wmv_ratio,25%_fd2wmv_ratio,50%_fd2wmv_ratio,75%_fd2wmv_ratio,max_fd2wmv_ratio,vehicle_measure_fdbus_ratio,count_fdbus_ratio,mean_fdbus_ratio,std_fdbus_ratio,min_fdbus_ratio,25%_fdbus_ratio,50%_fdbus_ratio,75%_fdbus_ratio,max_fdbus_ratio,vehicle_measure_fdcar_ratio,count_fdcar_ratio,mean_fdcar_ratio,std_fdcar_ratio,min_fdcar_ratio,25%_fdcar_ratio,50%_fdcar_ratio,75%_fdcar_ratio,max_fdcar_ratio,vehicle_measure_fdhgv_ratio,count_fdhgv_ratio,mean_fdhgv_ratio,std_fdhgv_ratio,min_fdhgv_ratio,25%_fdhgv_ratio,50%_fdhgv_ratio,75%_fdhgv_ratio,max_fdhgv_ratio,vehicle_measure_fdhgva3_ratio,count_fdhgva3_ratio,mean_fdhgva3_ratio,std_fdhgva3_ratio,min_fdhgva3_ratio,25%_fdhgva3_ratio,50%_fdhgva3_ratio,75%_fdhgva3_ratio,max_fdhgva3_ratio,vehicle_measure_fdhgva5_ratio,count_fdhgva5_ratio,mean_fdhgva5_ratio,std_fdhgva5_ratio,min_fdhgva5_ratio,25%_fdhgva5_ratio,50%_fdhgva5_ratio,75%_fdhgva5_ratio,max_fdhgva5_ratio,vehicle_measure_fdhgva6_ratio,count_fdhgva6_ratio,mean_fdhgva6_ratio,std_fdhgva6_ratio,min_fdhgva6_ratio,25%_fdhgva6_ratio,50%_fdhgva6_ratio,75%_fdhgva6_ratio,max_fdhgva6_ratio,vehicle_measure_fdhgvr2_ratio,count_fdhgvr2_ratio,mean_fdhgvr2_ratio,std_fdhgvr2_ratio,min_fdhgvr2_ratio,25%_fdhgvr2_ratio,50%_fdhgvr2_ratio,75%_fdhgvr2_ratio,max_fdhgvr2_ratio,vehicle_measure_fdhgvr3_ratio,count_fdhgvr3_ratio,mean_fdhgvr3_ratio,std_fdhgvr3_ratio,min_fdhgvr3_ratio,25%_fdhgvr3_ratio,50%_fdhgvr3_ratio,75%_fdhgvr3_ratio,max_fdhgvr3_ratio,vehicle_measure_fdhgvr4_ratio,count_fdhgvr4_ratio,mean_fdhgvr4_ratio,std_fdhgvr4_ratio,min_fdhgvr4_ratio,25%_fdhgvr4_ratio,50%_fdhgvr4_ratio,75%_fdhgvr4_ratio,max_fdhgvr4_ratio,vehicle_measure_fdlgv_ratio,count_fdlgv_ratio,mean_fdlgv_ratio,std_fdlgv_ratio,min_fdlgv_ratio,25%_fdlgv_ratio,50%_fdlgv_ratio,75%_fdlgv_ratio,max_fdlgv_ratio,vehicle_measure_fdpc_ratio,count_fdpc_ratio,mean_fdpc_ratio,std_fdpc_ratio,min_fdpc_ratio,25%_fdpc_ratio,50%_fdpc_ratio,75%_fdpc_ratio,max_fdpc_ratio,vehicle_measure_log_fd2wmv,count_log_fd2wmv,mean_log_fd2wmv,std_log_fd2wmv,min_log_fd2wmv,25%_log_fd2wmv,50%_log_fd2wmv,75%_log_fd2wmv,max_log_fd2wmv,vehicle_measure_log_fdall_mv,count_log_fdall_mv,mean_log_fdall_mv,std_log_fdall_mv,min_log_fdall_mv,25%_log_fdall_mv,50%_log_fdall_mv,75%_log_fdall_mv,max_lo

In [91]:
stats_unmelted_df.to_csv(os.getcwd()
                         + "/data/eco_stats_unmelted.csv"
                        )

Get the columns that contain the data

In [93]:
data_mask = [(("mean" in col) 
              | ("gdp_growth" in col)
              | ("CPI" in col)
              | ("earnings" in col)
              | ("unemployment" in col.lower())
             )
             & ("ratio" not in col)
             & ("log" not in col)
             & ("hgva" not in col)
             & ("hgvr" not in col)
             for col in stats_unmelted_df.columns.tolist()
            ]

data_columns = stats_unmelted_df.columns[data_mask].tolist()

data_columns

['mean_fd2wmv',
 'gdp_growth',
 'CPIH',
 'CPI',
 'Unemployment (%)',
 'average_weekly_earnings',
 'av_weekly_earnings_change',
 'unemployment_pct_change',
 'mean_fdall_mv',
 'mean_fdbus',
 'mean_fdcar',
 'mean_fdhgv',
 'mean_fdlgv',
 'mean_fdpc']

In [94]:
cats = traffic_df["rcat"].unique().tolist()

traffic_df["rcat"] == cats[1]

0      False
1       True
2      False
3      False
4      False
5      False
6      False
7      False
8      False
9      False
10     False
11     False
12     False
13     False
14      True
15     False
16     False
17     False
18     False
19     False
20     False
21     False
22     False
23     False
24     False
25     False
26     False
27     False
28     False
29      True
       ...  
143    False
144    False
145    False
146    False
147    False
148    False
149     True
150    False
151    False
152    False
153    False
154    False
155    False
156    False
157    False
158    False
159    False
160    False
161     True
162    False
163    False
164    False
165    False
166    False
167    False
168    False
169    False
170    False
171    False
172    False
Name: rcat, Length: 173, dtype: bool

In [95]:
#get the rows we need
mask = ((stats_unmelted_df["area_level"] == "nuts1") 
        & (stats_unmelted_df["area_name"] == "nat") 
       )
traffic_df = stats_unmelted_df[mask]

#we assume nan means no traffic...
traffic_df.replace([np.nan, -np.inf, np.inf], 0, inplace = True)

#scale for each road category
scaled_lst = []
for cat in traffic_df["rcat"].unique().tolist():

    df = traffic_df[traffic_df["rcat"] == cat]
    #and scale
    scaler = StandardScaler()
    scaled = scaler.fit_transform(df[data_columns].values)

    scaled_lst.append(pd.DataFrame(data = scaled, 
                                   columns = data_columns,
                                   index = df["year_label"]
                                  )
                     )
    scaled_lst[-1]["rcat"] = cat
    
scaled_df = pd.concat(scaled_lst)

In [96]:
HTML(traffic_df.head().to_html())

,vehicle_measure_fd2wmv,count_fd2wmv,mean_fd2wmv,std_fd2wmv,min_fd2wmv,25%_fd2wmv,50%_fd2wmv,75%_fd2wmv,max_fd2wmv,area_name,area_level,rcat,gdp,gdp_unit,gdp_growth,gdp_ratio,year_label,year,CPIH,CPI,Unemployment (%),average_weekly_earnings,av_weekly_earnings_change,unemployment_pct_change,vehicle_measure_fdall_mv,count_fdall_mv,mean_fdall_mv,std_fdall_mv,min_fdall_mv,25%_fdall_mv,50%_fdall_mv,75%_fdall_mv,max_fdall_mv,vehicle_measure_fdbus,count_fdbus,mean_fdbus,std_fdbus,min_fdbus,25%_fdbus,50%_fdbus,75%_fdbus,max_fdbus,vehicle_measure_fdcar,count_fdcar,mean_fdcar,std_fdcar,min_fdcar,25%_fdcar,50%_fdcar,75%_fdcar,max_fdcar,vehicle_measure_fdhgv,count_fdhgv,mean_fdhgv,std_fdhgv,min_fdhgv,25%_fdhgv,50%_fdhgv,75%_fdhgv,max_fdhgv,vehicle_measure_fdhgva3,count_fdhgva3,mean_fdhgva3,std_fdhgva3,min_fdhgva3,25%_fdhgva3,50%_fdhgva3,75%_fdhgva3,max_fdhgva3,vehicle_measure_fdhgva5,count_fdhgva5,mean_fdhgva5,std_fdhgva5,min_fdhgva5,25%_fdhgva5,50%_fdhgva5,75%_fdhgva5,max_fdhgva5,vehicle_measure_fdhgva6,count_fdhgva6,mean_fdhgva6,std_fdhgva6,min_fdhgva6,25%_fdhgva6,50%_fdhgva6,75%_fdhgva6,max_fdhgva6,vehicle_measure_fdhgvr2,count_fdhgvr2,mean_fdhgvr2,std_fdhgvr2,min_fdhgvr2,25%_fdhgvr2,50%_fdhgvr2,75%_fdhgvr2,max_fdhgvr2,vehicle_measure_fdhgvr3,count_fdhgvr3,mean_fdhgvr3,std_fdhgvr3,min_fdhgvr3,25%_fdhgvr3,50%_fdhgvr3,75%_fdhgvr3,max_fdhgvr3,vehicle_measure_fdhgvr4,count_fdhgvr4,mean_fdhgvr4,std_fdhgvr4,min_fdhgvr4,25%_fdhgvr4,50%_fdhgvr4,75%_fdhgvr4,max_fdhgvr4,vehicle_measure_fdlgv,count_fdlgv,mean_fdlgv,std_fdlgv,min_fdlgv,25%_fdlgv,50%_fdlgv,75%_fdlgv,max_fdlgv,vehicle_measure_fdpc,count_fdpc,mean_fdpc,std_fdpc,min_fdpc,25%_fdpc,50%_fdpc,75%_fdpc,max_fdpc,vehicle_measure_fd2wmv_ratio,count_fd2wmv_ratio,mean_fd2wmv_ratio,std_fd2wmv_ratio,min_fd2wmv_ratio,25%_fd2wmv_ratio,50%_fd2wmv_ratio,75%_fd2wmv_ratio,max_fd2wmv_ratio,vehicle_measure_fdbus_ratio,count_fdbus_ratio,mean_fdbus_ratio,std_fdbus_ratio,min_fdbus_ratio,25%_fdbus_ratio,50%_fdbus_ratio,75%_fdbus_ratio,max_fdbus_ratio,vehicle_measure_fdcar_ratio,count_fdcar_ratio,mean_fdcar_ratio,std_fdcar_ratio,min_fdcar_ratio,25%_fdcar_ratio,50%_fdcar_ratio,75%_fdcar_ratio,max_fdcar_ratio,vehicle_measure_fdhgv_ratio,count_fdhgv_ratio,mean_fdhgv_ratio,std_fdhgv_ratio,min_fdhgv_ratio,25%_fdhgv_ratio,50%_fdhgv_ratio,75%_fdhgv_ratio,max_fdhgv_ratio,vehicle_measure_fdhgva3_ratio,count_fdhgva3_ratio,mean_fdhgva3_ratio,std_fdhgva3_ratio,min_fdhgva3_ratio,25%_fdhgva3_ratio,50%_fdhgva3_ratio,75%_fdhgva3_ratio,max_fdhgva3_ratio,vehicle_measure_fdhgva5_ratio,count_fdhgva5_ratio,mean_fdhgva5_ratio,std_fdhgva5_ratio,min_fdhgva5_ratio,25%_fdhgva5_ratio,50%_fdhgva5_ratio,75%_fdhgva5_ratio,max_fdhgva5_ratio,vehicle_measure_fdhgva6_ratio,count_fdhgva6_ratio,mean_fdhgva6_ratio,std_fdhgva6_ratio,min_fdhgva6_ratio,25%_fdhgva6_ratio,50%_fdhgva6_ratio,75%_fdhgva6_ratio,max_fdhgva6_ratio,vehicle_measure_fdhgvr2_ratio,count_fdhgvr2_ratio,mean_fdhgvr2_ratio,std_fdhgvr2_ratio,min_fdhgvr2_ratio,25%_fdhgvr2_ratio,50%_fdhgvr2_ratio,75%_fdhgvr2_ratio,max_fdhgvr2_ratio,vehicle_measure_fdhgvr3_ratio,count_fdhgvr3_ratio,mean_fdhgvr3_ratio,std_fdhgvr3_ratio,min_fdhgvr3_ratio,25%_fdhgvr3_ratio,50%_fdhgvr3_ratio,75%_fdhgvr3_ratio,max_fdhgvr3_ratio,vehicle_measure_fdhgvr4_ratio,count_fdhgvr4_ratio,mean_fdhgvr4_ratio,std_fdhgvr4_ratio,min_fdhgvr4_ratio,25%_fdhgvr4_ratio,50%_fdhgvr4_ratio,75%_fdhgvr4_ratio,max_fdhgvr4_ratio,vehicle_measure_fdlgv_ratio,count_fdlgv_ratio,mean_fdlgv_ratio,std_fdlgv_ratio,min_fdlgv_ratio,25%_fdlgv_ratio,50%_fdlgv_ratio,75%_fdlgv_ratio,max_fdlgv_ratio,vehicle_measure_fdpc_ratio,count_fdpc_ratio,mean_fdpc_ratio,std_fdpc_ratio,min_fdpc_ratio,25%_fdpc_ratio,50%_fdpc_ratio,75%_fdpc_ratio,max_fdpc_ratio,vehicle_measure_log_fd2wmv,count_log_fd2wmv,mean_log_fd2wmv,std_log_fd2wmv,min_log_fd2wmv,25%_log_fd2wmv,50%_log_fd2wmv,75%_log_fd2wmv,max_log_fd2wmv,vehicle_measure_log_fdall_mv,count_log_fdall_mv,mean_log_fdall_mv,std_log_fdall_mv,min_log_fdall_mv,25%_log_fdall_mv,50%_log_fdall_mv,75%_log_fdall_mv,max_lo

In [97]:
HTML(scaled_df.head().to_html())

,mean_fd2wmv,gdp_growth,CPIH,CPI,Unemployment (%),average_weekly_earnings,av_weekly_earnings_change,unemployment_pct_change,mean_fdall_mv,mean_fdbus,mean_fdcar,mean_fdhgv,mean_fdlgv,mean_fdpc,rcat
year_label,,,,,,,,,,,,,,,
2011,-0.576002,-0.067851,1.570187,2.004465e+00,1.413919,0.657173,0.000000,0.249513,-0.436878,0.275725,-0.472571,-0.906208,0.110025,0.796051,PR
2005,0.805089,0.762331,-1.424687,-1.822241e-01,-1.167235,-1.130950,1.076297,-0.020793,0.056203,-0.860004,0.548495,1.048769,-0.905045,-1.303126,PR
2015,-0.817267,0.347240,-1.109437,-2.095577e+00,-0.697935,1.352555,0.179383,-1.102016,1.808169,-0.589287,1.099636,-0.509542,2.096851,1.004379,PR
2003,1.899652,0.866104,-1.424687,-8.200084e-01,-1.010802,-1.925671,0.000000,-0.291099,-1.836501,2.663867,-1.428698,0.313871,-1.839570,-0.943252,PR
2007,0.493381,0.399126,0.466812,-4.046188e-16,-0.776151,-0.212053,1.435063,-0.155946,1.399407,0.162538,1.297938,1.501794,0.413921,-1.162636,PR


In [98]:
traffic_df.to_csv(os.getcwd() + "/data/traffic_data_all_roads.csv")
scaled_df.to_csv(os.getcwd() + "/data/scaled_traffic_data_all_roads.csv")

This is the data use for the cross correlations

In [99]:
corr_df = traffic_df[data_columns + ["year_label"]]
#arrange df by year
corr_df.set_index("year_label", inplace = True)
corr_df.sort_index(inplace = True)

corr_df.to_csv(os.getcwd() + "/data/cross_correlations_data_all_roads.csv")
corr_df.head()

/home/eddr/.local/share/virtualenvs/report-GvhoWphm/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,mean_fd2wmv,gdp_growth,CPIH,CPI,Unemployment (%),average_weekly_earnings,av_weekly_earnings_change,unemployment_pct_change,mean_fdall_mv,mean_fdbus,mean_fdcar,mean_fdhgv,mean_fdlgv,mean_fdpc
year_label,,,,,,,,,,,,,,
2003,189.764529,3.3,0.0,1.4,5.0,351.0,11.0,-0.2,24074.915331,127.716433,18810.777555,2209.507014,2737.149800,8.517034
2003,219.458572,3.3,0.0,1.4,5.0,351.0,11.0,-0.2,17708.208595,213.464347,14268.918690,1032.794043,1973.572988,97.548840
2003,23.317355,3.3,0.0,1.4,5.0,351.0,11.0,-0.2,2323.611570,30.224793,1980.170248,50.158678,239.740496,27.624793
2003,20.774584,3.3,0.0,1.4,5.0,351.0,11.0,-0.2,2014.319213,21.856278,1694.381241,54.623298,222.683812,17.937973
2003,122.196319,3.3,0.0,1.4,5.0,351.0,11.0,-0.2,12060.714724,197.558282,10374.377301,278.134969,1088.447853,135.828221


Dataset for correlations visualisation

In [11]:
vis_lst = []
for cat in stats_unmelted_df["rcat"].unique().tolist():
    #columns that contain data
    data_mask = [col 
             for col in stats_unmelted_df.columns.tolist()
             if(("ratio" not in col)
                 & ("hgvr" not in col)
                 & ("measure" not in col)
                 & ("max" not in col)
                 & ("min" not in col)
                 & ("%" not in col)
                 & ("std" not in col)
                 & ("count" not in col)
                 & ("unit" not in col)
               )
            ]

    df = stats_unmelted_df[data_mask]
    df.set_index("year_label", inplace = True)
    df.drop("year", axis = 1, inplace = True)
    vis_lst.append(df)

vis_df = pd.concat(vis_lst)
HTML(vis_df.head().to_html())

/home/eddr/.bashrc-anaconda3.bak/lib/python3.5/site-packages/ipykernel/__main__.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,mean_fd2wmv,area_name,area_level,rcat,gdp,gdp_growth,CPIH,CPI,average_weekly_earnings,av_weekly_earnings_change,unemployment_pct_change,mean_fdall_mv,mean_fdbus,mean_fdcar,mean_fdhgv,mean_fdhgva3,mean_fdhgva5,mean_fdhgva6,mean_fdlgv,mean_fdpc,mean_log_fd2wmv,mean_log_fdall_mv,mean_log_fdbus,mean_log_fdcar,mean_log_fdhgv,mean_log_fdhgva3,mean_log_fdhgva5,mean_log_fdhgva6,mean_log_fdlgv,mean_log_fdpc
year_label,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2011,99.717961,nat,nuts1,PR,1729121.0,1.5,3.8,4.5,455.0,11.0,0.2,11554.632907,83.035398,9335.433956,518.935595,27.402163,75.258112,85.881514,1517.509997,25.716159,4.206119,9.003183,4.039661,8.769538,5.774744,2.738523,3.364775,3.515354,7.014576,2.562238
2011,298.106322,nat,nuts1,TM,1729121.0,1.5,3.8,4.5,455.0,11.0,0.2,72523.555556,318.524904,55172.217433,7468.451149,313.326628,1785.848659,2685.389847,9266.255747,0.000000,5.401164,10.981366,5.510957,10.704581,8.576902,5.331215,6.972889,7.304020,8.903942,NaN
2011,270.510982,nat,nuts1,PU,1729121.0,1.5,3.8,4.5,455.0,11.0,0.2,19339.268449,363.962231,15709.632772,595.098547,21.201046,47.893202,62.462754,2400.063916,254.307844,4.909887,9.698336,5.320681,9.484507,5.982574,2.480183,2.914966,3.088334,7.601718,4.531359
2011,161.170022,nat,nuts1,TR,1729121.0,1.5,3.8,4.5,455.0,11.0,0.2,25215.234899,116.944072,19413.483781,2142.809843,103.916667,482.131432,664.616331,3380.827181,7.592841,4.650228,9.721648,4.461427,9.447724,7.109951,4.016201,5.262564,5.578707,7.739932,1.644697
2011,210.409241,nat,nuts1,TU,1729121.0,1.5,3.8,4.5,455.0,11.0,0.2,34200.458746,189.300330,27151.330033,2118.742574,86.976898,389.561056,564.376238,4530.676568,44.221122,5.047091,10.199580,4.959478,9.961111,7.227431,3.922960,5.122767,5.431130,8.188123,3.037261


In [12]:
def convert_col_names(df, col_key):
    for key in col_key:   
        df.columns = ["log(" + key[1] + ")" 
                      if (("log" in col) & (key[0] in col))
                      else col
                      for col in df.columns.tolist()
                     ]

        df.columns = [key[1]
                      if  (key[0] in col)
                      else col
                      for col in df.columns.tolist()
                     ]

    return df

In [14]:
col_key = [("fd2wmv", "Motorbikes and Scooters"),
             ("gdp_growth", "GDP Growth"),
             ("av_weekly_earnings_change", "Change in weekly earnings (£)"),
             ("unemployment_pct_change", "Change in unemployment (% pts)"),
             ("fdall_mv","All Motor Vehicles"),
             ("fdbus", "Buses and Coaches"),
             ("fdcar", "Cars and Taxis"),
             ("fdhgv", "All HGVs"),
             ("fdlgv", "LGVs"),
             ("fdpc", "Pedal Cycles"),
             ("rcat", "Road Category")]

In [15]:
vis_df = convert_col_names(vis_df,col_key)

#select only national level data
mask = (vis_df["area_level"] == "nuts1") & (vis_df["area_name"] == "nat")

vis_df = vis_df[mask]


HTML(vis_df.head().to_html())

,Motorbikes and Scooters,area_name,area_level,Road Category,gdp,GDP Growth,CPIH,CPI,average_weekly_earnings,Change in weekly earnings (£),Change in unemployment (% pts),All Motor Vehicles,Buses and Coaches,Cars and Taxis,All HGVs,All HGVs,All HGVs,All HGVs,LGVs,Pedal Cycles,log(Motorbikes and Scooters),log(All Motor Vehicles),log(Buses and Coaches),log(Cars and Taxis),log(All HGVs),log(All HGVs),log(All HGVs),log(All HGVs),log(LGVs),log(Pedal Cycles)
year_label,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2011,99.717961,nat,nuts1,PR,1729121.0,1.5,3.8,4.5,455.0,11.0,0.2,11554.632907,83.035398,9335.433956,518.935595,27.402163,75.258112,85.881514,1517.509997,25.716159,4.206119,9.003183,4.039661,8.769538,5.774744,2.738523,3.364775,3.515354,7.014576,2.562238
2011,298.106322,nat,nuts1,TM,1729121.0,1.5,3.8,4.5,455.0,11.0,0.2,72523.555556,318.524904,55172.217433,7468.451149,313.326628,1785.848659,2685.389847,9266.255747,0.000000,5.401164,10.981366,5.510957,10.704581,8.576902,5.331215,6.972889,7.304020,8.903942,NaN
2011,270.510982,nat,nuts1,PU,1729121.0,1.5,3.8,4.5,455.0,11.0,0.2,19339.268449,363.962231,15709.632772,595.098547,21.201046,47.893202,62.462754,2400.063916,254.307844,4.909887,9.698336,5.320681,9.484507,5.982574,2.480183,2.914966,3.088334,7.601718,4.531359
2011,161.170022,nat,nuts1,TR,1729121.0,1.5,3.8,4.5,455.0,11.0,0.2,25215.234899,116.944072,19413.483781,2142.809843,103.916667,482.131432,664.616331,3380.827181,7.592841,4.650228,9.721648,4.461427,9.447724,7.109951,4.016201,5.262564,5.578707,7.739932,1.644697
2011,210.409241,nat,nuts1,TU,1729121.0,1.5,3.8,4.5,455.0,11.0,0.2,34200.458746,189.300330,27151.330033,2118.742574,86.976898,389.561056,564.376238,4530.676568,44.221122,5.047091,10.199580,4.959478,9.961111,7.227431,3.922960,5.122767,5.431130,8.188123,3.037261


Good, lets save this dataset

In [16]:
vis_df.to_csv(os.getcwd() + "/data/correlations_vis_all_roads.csv")